In [ ]:
from keras.layers import Dense, Flatten, regularizers
from keras.layers import Conv2D
import keras.layers
import keras
from tensorflow.keras import layers

ImportError: ignored

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization,AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda, Flatten, Activation
trunk=64
blocks=64
filters=64
import tensorflow as tf

from tensorflow.keras import regularizers

def getModel ():
    input = keras.Input(shape=(19, 19, planes), name='board')
    x = Conv2D(trunk, 1, padding='same', kernel_regularizer=regularizers.l2(0.0001))(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    for i in range (blocks):
        x = bottleneck_block (x, filters, trunk)
    policy_head = Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
    policy_head = Flatten()(policy_head)
    policy_head = Activation('softmax', name='policy')(policy_head)
    value_head = GlobalAveragePooling2D()(x)
    value_head = Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
    value_head = Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

    model = keras.Model(inputs=input, outputs=[policy_head, value_head])
 
    return model

In [ ]:
def bottleneck_block(tensor, expand=64, squeeze=16):
    x = gconv(tensor, channels=expand, groups=4)
    x = BatchNormalization()(x)
    x = ReLU()(x)
 
    x = channel_shuffle(x, groups=4)
    x = DepthwiseConv2D(kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
 
    x = gconv(x, channels=squeeze, groups=4)
    x = BatchNormalization()(x)
 
    x = Add()([tensor, x])
    output = ReLU()(x)
    return output

In [ ]:
def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []
 
    for i in range(groups):
        group_tensor = tensor[:, :, :, i * group_ch: (i+1) * group_ch]
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)
 
    output = Concatenate()(groups_list)
    return output

In [ ]:
def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups
 
    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

In [ ]:
my_model=getModel()